<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/legal_longformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [5]:
!pip install transformers.modeling_longformer 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement transformers.modeling_longformer (from versions: none)
ERROR: No matching distribution found for transformers.modeling_longformer


In [3]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

model = AutoModelForPreTraining.from_pretrained("nlpaueb/legal-bert-base-uncased")

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/217k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

In [6]:
import logging
import os
import math
import copy
import torch
from dataclasses import dataclass, field
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer
from transformers import TrainingArguments, HfArgumentParser
from transformers import LongformerConfig, LongformerModel

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [7]:
class LBERTLongSelfAttention(LongformerSelfAttention):
    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        output_attentions=False,
    ):
        return super().forward(hidden_states, attention_mask=attention_mask, output_attentions=output_attentions)


class LBERTLongForMaskedLM(model):
    def __init__(self, config):
        super().__init__(config)
        for i, layer in enumerate(self.lbert.encoder.layer):
            # replace the `modeling_bert.BertSelfAttention` object with `LongformerSelfAttention`
            layer.attention.self = LBERTLongSelfAttention(config, layer_id=i)

NameError: ignored